In [1]:
import numpy as np
import pandas as pd 
import ipaddress
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

In [2]:
creditcard_data = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\Creditcard_cleaned_Data.csv")
fraud_data = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\fraud_cleaned_Data.csv")
ip_to_country = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\ip_to_country_cleaned_Data.csv")

In [ ]:
creditcard_data.sample(7)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
217688,141325.0,-1.829458,-0.908104,-1.686152,-1.141003,2.821679,3.252671,0.398063,1.344621,-0.553375,-0.843946,-0.409692,0.163265,-0.231043,0.817872,0.064508,-0.067842,-0.295870,-0.593487,-0.211816,0.808813,0.061527,-0.733832,0.662215,0.666015,-0.429128,-0.036453,0.196760,0.010772,247.95,0
57172,47900.0,-0.162374,0.883811,0.788843,-0.512012,0.549094,-0.699019,0.729215,-0.033776,-0.599304,-0.613595,1.096415,0.937488,0.706121,-0.344741,-0.491352,0.906900,-0.753120,0.333684,-0.454482,-0.105864,-0.059066,-0.214043,0.162750,-0.040006,-1.248761,-0.261916,0.191912,0.234848,1.98,0
223938,143957.0,0.103644,1.525405,0.563768,4.074760,1.666980,2.108204,0.573568,0.367916,-2.379389,1.439533,-0.379337,-0.407205,0.692751,0.488209,1.672505,-1.126774,0.842806,-1.161992,-0.139258,0.092371,0.436392,1.457316,-0.087321,-0.307026,-0.852544,0.550499,0.279929,0.211620,2.28,0
194994,131179.0,0.447100,0.615680,-1.001129,1.109371,1.300932,-0.543347,0.935549,-0.393686,-0.587051,0.192032,-0.995271,0.213512,0.921034,0.505670,0.886124,-2.058155,0.684746,0.298656,2.582019,0.305530,0.742442,2.471164,-0.270451,0.687763,-0.570462,0.530946,0.308917,0.285922,22.03,0
189297,128637.0,1.430522,-3.065353,-0.534807,-1.083244,-2.307702,0.241679,-1.344188,0.129560,-0.466287,1.374734,-0.182803,-0.512549,-0.830920,-0.583413,-1.598176,-0.277238,0.483494,0.566133,0.441225,0.297747,-0.143328,-0.722803,0.011867,-0.388354,-0.684121,-0.403237,-0.023311,0.019125,412.94,0
190595,129198.0,2.234480,-1.443576,-1.305660,-1.499699,-1.299041,-1.290668,-0.813523,-0.342779,-1.347491,1.695505,-1.038180,-1.614366,-0.843407,0.131240,0.408859,-0.576540,0.522320,0.168355,-0.438952,-0.438484,0.103291,0.632811,0.051413,-0.028789,0.001583,0.079792,-0.023908,-0.055147,60.00,0
85494,60949.0,-0.436492,0.053487,2.233529,-0.282395,-0.881124,-0.081431,0.223053,-0.016097,1.038061,-0.535823,-0.407567,-0.146222,-1.334943,-0.593909,-0.238922,-0.714960,0.542276,-0.966500,0.055331,-0.129514,-0.082723,0.148030,0.153688,0.649756,-1.188783,0.752387,-0.110656,-0.065851,59.14,0


In [4]:
fraud_data.sample(7)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour
133471,373014,2015-04-07 15:36:50,2015-05-18 23:41:07,24.0,EYSDZZTFAUZFU,SEO,Opera,M,41,3.919820e+09,0,23,0,4,7,15,5,18,23
133546,94302,2015-07-09 08:31:01,2015-08-23 09:59:40,32.0,ASZXTCKEHYTSW,Direct,Chrome,M,31,4.061834e+08,0,9,6,7,9,8,8,23,9
17457,17291,2015-07-31 21:59:15,2015-09-11 04:33:13,47.0,MGLFJKGJHLVTE,SEO,IE,F,19,7.144500e+08,0,4,4,7,31,21,9,11,4
63762,210313,2015-08-14 12:08:21,2015-09-19 15:17:49,72.0,SHVJFSJYWRGHV,SEO,FireFox,M,28,2.678588e+09,0,15,5,8,14,12,9,19,15
91815,116718,2015-06-27 04:56:19,2015-09-04 02:24:58,41.0,HBMUIJZFYBAUL,SEO,Chrome,F,21,8.269279e+08,0,2,4,6,27,4,9,4,2
129180,201710,2015-04-05 05:02:32,2015-04-17 09:59:02,36.0,MWALMUQDIKEOO,Ads,FireFox,M,34,6.352234e+08,0,9,4,4,5,5,4,17,9
95155,295731,2015-03-25 13:37:20,2015-04-29 07:21:15,20.0,VLNUUBWETEBBK,SEO,FireFox,M,37,2.557601e+09,0,7,2,3,25,13,4,29,7


In [5]:
ip_to_country.sample(7)

,lower_bound_ip_address,upper_bound_ip_address,country
70605,3.231493e+09,3231493119,United States
130794,3.495266e+09,3495268351,United States
129215,3.479011e+09,3479019519,United States
132057,3.517096e+09,3517100031,Canada
120169,3.423201e+09,3423202303,United States
98969,3.334390e+09,3334390015,United States
85021,3.261822e+09,3261822719,United Kingdom


In [6]:
# Assuming fraud_data['ip_address'] is already numeric (integer representation)
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].astype(float).astype(int)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].astype(float).astype(int)

# Now, define a function to map IP to a country
def map_country(ip, ip_df):
    result = ip_df[(ip_df['lower_bound_ip_address'] <= ip) & (ip_df['upper_bound_ip_address'] >= ip)]
    return result['country'].values[0] if len(result) > 0 else np.nan

# Apply the function to map the IP addresses to their respective countries
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))


In [7]:
import ipaddress

# Convert the ip_address column in fraud_data to integers
fraud_data['ip_address'] = fraud_data['ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)

# Convert the lower and upper IP address bounds in the ip_to_country DataFrame to integers
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)

# Now map the country based on IP address
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))

In [8]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,2,5,2,24,22,4,18,2,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,0,6,7,20,6,8,1,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,18,3,1,1,18,1,1,18,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,13,0,4,28,21,5,4,13,NaN
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,18,2,7,21,7,9,9,18,United States


In [9]:
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))

creditcard_data.rename(columns={'Class': 'Class_creditcard'}, inplace=True)
fraud_data.rename(columns={'class': 'class_fraud'}, inplace=True)

missing_cols_in_fraud = set(creditcard_data.columns) - set(fraud_data.columns)
missing_cols_in_credit = set(fraud_data.columns) - set(creditcard_data.columns)

for col in missing_cols_in_fraud:
    fraud_data[col] = np.nan
for col in missing_cols_in_credit:
    creditcard_data[col] = np.nan

combined_data = pd.concat([fraud_data, creditcard_data], ignore_index=True)

# Inspect the merged dataset
combined_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,Time,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,2.0,5.0,2.0,24.0,22.0,4.0,18.0,2.0,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,1.0,0.0,6.0,7.0,20.0,6.0,8.0,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,18.0,3.0,1.0,1.0,18.0,1.0,1.0,18.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,13.0,0.0,4.0,28.0,21.0,5.0,4.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,18.0,2.0,7.0,21.0,7.0,9.0,9.0,18.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
combined_data.sample(10)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,Time,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19
394949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152470.0,0.499059,0.794821,0.258177,-1.832858,-0.052554,0.585574,-0.063853,0.257424,-0.447052,-0.034481,-0.740825,-0.342310,2.143771,0.016171,-0.514900,2.089796,0.156799,-0.070413,-0.190143,0.171589,0.372042,0.765219,84.86,-1.983416,0.581869,3.033143,-3.263904,0.0,-1.141858,0.740363
372473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142856.0,-0.915829,1.396096,-0.286767,3.611441,-0.035718,-0.810462,-1.278823,0.035348,0.277678,0.340553,-1.126101,-0.932823,0.861310,-0.363473,0.328243,1.953802,-0.190030,-0.036417,1.082906,0.845194,-0.163228,0.418053,7.30,-0.810541,-0.109885,0.246595,-0.838164,0.0,0.691287,-1.209364
110784,191107.0,2015-01-19 03:20:57,2015-01-28 16:03:55,11.0,HLGRMTIDPKHBI,Direct,Safari,M,27.0,3.493392e+09,0.0,16.0,2.0,1.0,19.0,3.0,1.0,28.0,16.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72731.0,0.110060,-0.647900,-0.212755,0.147671,0.028343,-0.604394,-0.193264,-0.335754,0.558636,0.264626,-0.617593,0.781050,-0.117674,0.068297,0.400130,-1.011201,0.009759,0.006644,1.097603,0.464618,-0.060024,0.353249,14.97,-0.209357,0.261428,-1.082484,0.880108,0.0,1.231522,-0.078152
217785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52286.0,0.489444,-0.344581,-0.024636,1.291878,0.012140,-0.444218,1.296026,-0.329059,-0.380271,-0.140026,-0.006537,0.933054,-0.033509,-0.422760,0.019182,0.888742,0.041686,0.057185,-1.956834,0.754942,0.328909,-0.491165,140.59,0.788961,0.201052,-0.064018,-0.276897,0.0,-0.075472,-0.587346
287129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81747.0,-0.414627,1.157920,-0.524056,-0.331571,0.691717,-0.851659,-0.673659,-1.065626,0.160211,0.095222,0.599622,0.979758,0.452211,0.146500,0.630007,-1.835868,0.602332,0.460605,1.162767,0.501214,0.286353,-0.509720,1.29,-0.072709,-0.078543,-0.694680,0.359247,0.0,1.717783,0.115057
248105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66185.0,-0.129910,1.488627,-0.116924,-2.499780,0.055823,-0.055789,-0.278283,-0.370603,-1.048611,-0.031233,-1.736318,1.020330,0.695935,-0.087609,-1.838142,1.566235,-0.204238,0.019554,0.600902,0.172956,0.504343,0.996914,6.00,0.743245,0.856761,3.453596,-0.767050,0.0,-1.316768,-0.067603
278865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78645.0,-0.729312,-0.401810,-0.201512,0.295577,-0.090416,-0.848638,-0.382694,0.127596,0.274530,0.319203,-0.578225,0.109149,1.335389,-0.987559,1.064244,-1.047281,0.296328,-0.044250,0.381719,0.007103,-0.311752,0.323188,69.10,-0.616573,-0.404459,-1.550255,1.328416,0.0,0.015265,-0.617896
44691,182503.0,2015-03-29 22:22:18,2015-05-15 14:57:39,34.0,DHFOQDIVVUZHN,SEO,Safari,M,34.0,1.477243e+09,0.0,14.0,4.0,3.0,29.0,22.0,5.0,15.0,14.0,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96030.0,-0.727182,1.374104,-0.261549,-1.444010,-0.383696,0.773713,1.624377,1.300759,-2.453986,-0.247492,2.278260,-1.575211,2.346656,-0.223652,1.954264,-1.779307,-0.202939,0.393406,1.034510,-0.375655,-0.459606,0.278380,52.79,0.332289,-1.202928,-1.084623,-1.314348,0.0,0.401253,-0.404891


In [11]:
# Drop columns with more than 65% missing values
combined_data = combined_data.dropna(thresh=combined_data.shape[0] * 0.20, axis=1)


In [ ]:
# Example of filling missing values
combined_data['country'].fillna('Unknown', inplace=True)  # For categorical columns

C:\Users\ephre\AppData\Local\Temp\ipykernel_30464\3989163506.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['country'].fillna('Unknown', inplace=True)  # For categorical columns


In [13]:
# Fill missing values in numerical columns with the mean
combined_data['age'] = combined_data['age'].fillna(combined_data['age'].mean())

# Fill missing values in categorical columns with the mode
combined_data['sex'] = combined_data['sex'].fillna(combined_data['sex'].mode()[0])
combined_data['browser'] = combined_data['browser'].fillna(combined_data['browser'].mode()[0])
combined_data['Amount'] = combined_data['Amount'].fillna(combined_data['Amount'].mode()[0])


In [14]:
numerical_columns = ["purchase_value", "age", "class_fraud", "Class_creditcard", "V3", "V27", "V14", "V18", "V10", "V21", "V8", "V23", "V1", "V5", "V19", "V22", "V17", "V16", "V9", "V12", "V2", "V26", "V15", "V7", "V24", "V13", "V25", "V20", "V11", "V28", "V4", "V6", "purchase_month", "purchase_day", "purchase_hour"]

# Apply mean filling to all numerical columns
for col in numerical_columns:
    combined_data[col] = combined_data[col].fillna(combined_data[col].mean())

# Check the resulting DataFrame
combined_data.head()


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,Time,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,2.0,5.0,2.0,24.0,22.0,4.0,18.0,2.0,Japan,NaN,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,1.0,0.0,6.0,7.0,20.0,6.0,8.0,1.0,United States,NaN,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,18.0,3.0,1.0,1.0,18.0,1.0,1.0,18.0,United States,NaN,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,13.0,0.0,4.0,28.0,21.0,5.0,4.0,13.0,Unknown,NaN,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,18.0,2.0,7.0,21.0,7.0,9.0,9.0,18.0,United States,NaN,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264


In [15]:
# List of columns you want to drop
columns_to_drop = ['user_id', 'signup_time', 'purchase_time', 'device_id', 'source', 'hour_of_day', 'day_of_week', 'signup_month', 'signup_day', 'signup_hour', 'Time']

# Drop the columns from the DataFrame
combined_data = combined_data.drop(columns=columns_to_drop)

# Check the resulting DataFrame
combined_data.head()


,purchase_value,browser,sex,age,ip_address,class_fraud,purchase_month,purchase_day,purchase_hour,country,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19
0,34.0,Chrome,M,39.0,7.327584e+08,0.0,4.0,18.0,2.0,Japan,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
1,16.0,Chrome,F,53.0,3.503114e+08,0.0,6.0,8.0,1.0,United States,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
2,15.0,Opera,M,53.0,2.621474e+09,1.0,1.0,1.0,18.0,United States,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
3,44.0,Safari,M,41.0,3.840542e+09,0.0,5.0,4.0,13.0,Unknown,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264
4,39.0,Safari,M,45.0,4.155831e+08,0.0,9.0,9.0,18.0,United States,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.000264


In [16]:
# combined_data = combined_data.dropna()

In [17]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434838 entries, 0 to 434837
Data columns (total 40 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   purchase_value    434838 non-null  float64
 1   browser           434838 non-null  object 
 2   sex               434838 non-null  object 
 3   age               434838 non-null  float64
 4   ip_address        151112 non-null  float64
 5   class_fraud       434838 non-null  float64
 6   purchase_month    434838 non-null  float64
 7   purchase_day      434838 non-null  float64
 8   purchase_hour     434838 non-null  float64
 9   country           434838 non-null  object 
 10  V17               434838 non-null  float64
 11  V10               434838 non-null  float64
 12  V21               434838 non-null  float64
 13  V4                434838 non-null  float64
 14  V27               434838 non-null  float64
 15  V22               434838 non-null  float64
 16  V11               43

In [18]:
# Categorical columns to encode
categorical_columns = ['browser', 'sex', 'country']

# Apply One-Hot Encoding
combined_data = pd.get_dummies(combined_data, columns=categorical_columns, drop_first=True)

# Check the result
combined_data.head()


,purchase_value,age,ip_address,class_fraud,purchase_month,purchase_day,purchase_hour,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,34.0,39.0,7.327584e+08,0.0,4.0,18.0,2.0,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.00026

In [19]:
boolean_columns = combined_data.select_dtypes(include="bool").columns

combined_data[boolean_columns] = combined_data[boolean_columns].astype(int)

In [20]:
combined_data.head()

,purchase_value,age,ip_address,class_fraud,purchase_month,purchase_day,purchase_hour,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,34.0,39.0,7.327584e+08,0.0,4.0,18.0,2.0,0.00017,-0.001441,-0.000371,-0.002966,0.001763,-0.000015,0.000202,0.000252,-0.000715,0.000198,-0.001596,0.001043,0.001828,0.000149,0.001801,0.005917,0.000187,0.000547,0.000603,0.001162,-0.000232,0.000214,1.0,0.001515,-0.000854,-0.001139,0.001613,0.001667,-0.004135,-0.00026

In [34]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

# 1. Filter numeric columns
numeric_columns = combined_data.select_dtypes(include=['float64', 'int64']).columns

# 2. Initialize the StandardScaler
scaler = StandardScaler()

# 3. Fit and transform only the numeric columns
scaled_data = scaler.fit_transform(combined_data[numeric_columns])

# 4. Create a DataFrame with the scaled numeric columns
scaled_data_df = pd.DataFrame(scaled_data, columns=numeric_columns)

# 5. Replace the original numeric columns in `combined_data` with the scaled columns
combined_data[numeric_columns] = scaled_data_df

# Now all the numeric columns in combined_data are standardized


In [40]:
combined_data.sample(7)

,purchase_value,age,ip_address,class_fraud,purchase_month,purchase_day,purchase_hour,V17,V10,V21,V4,V27,V22,V11,V14,V12,V23,V9,V15,V5,V26,V7,V1,V20,V28,V13,V16,V25,V24,Amount,V18,V8,V6,V3,Class_creditcard,V2,V19,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
346823,-3.658113e-17,-3.742861e-17,NaN,-8.591791e-18,-4.951493e-17,-1.152796e-17,1.074964e-17,6.831485e-02,8.653195e-01,5.626423e-01,-1.655637e+00,8.221147e-02,1.533924e+00,-5.731748e-02,-3.912962e-01,-5.290401e-01,3.252171e-01,-3.426369e-02,7.884252e-01,1.162799e+00,-1.763044e-01,-1.629163e+00,1.347904e+00,1.1850

In [37]:
scaler_MinMax = MinMaxScaler()
combined_data[numerical_columns] = scaler.fit_transform(combined_data[numerical_columns])